In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Analytic continuation

:::{note}

Improvements to analytic continuation in AmpForm are currently being developed in {doc}`compwa-report:003/index` and {doc}`compwa-report:004/index`.

:::

Analytic continuation allows one to handle resonances just below threshold ($m_0 < m_a + m_b$  in Eq. {eq}`relativistic_breit_wigner_with_ff`). In practice, this entails using a specific function for $\rho$ in Eq. {eq}`EnergyDependentWidth`.

## Definitions

Three usual choices for $\rho$ are the following:

In [ ]:
import warnings
from typing import Any

import sympy as sp
from IPython.display import Math
from sympy.printing.latex import LatexPrinter

from ampform.dynamics import DispersionIntegral
from ampform.dynamics.phasespace import BreakupMomentum
from ampform.io import aslatex
from ampform.sympy import unevaluated

warnings.filterwarnings("ignore")

### 1) Break-up momentum

The {func}`~sympy.functions.elementary.miscellaneous.sqrt` or {class}`.ComplexSqrt` of {class}`.BreakupMomentumSquared`:

In [ ]:
from ampform.dynamics import BreakupMomentumSquared

s, m_a, m_b, L = sp.symbols("s, m_a, m_b, L", nonnegative=True)
q_squared = BreakupMomentumSquared(s, m_a, m_b)
Math(aslatex({q_squared: q_squared.evaluate()}))

### 2) 'Normal' phase space factor

The 'normal' {class}`.PhaseSpaceFactor` (the denominator makes the difference to {eq}`EnergyDependentWidth`!):

In [ ]:
from ampform.dynamics import PhaseSpaceFactor

rho = PhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho: rho.evaluate()}))

### 3) 'Complex' phase space factor

A {class}`.PhaseSpaceFactorComplex` that uses {class}`.ComplexSqrt`:

In [ ]:
from ampform.dynamics import PhaseSpaceFactorComplex

rho_c = PhaseSpaceFactorComplex(s, m_a, m_b)
Math(aslatex({rho_c: rho_c.evaluate()}))

### 4) 'Analytic continuation' of the phase space factor

The following 'case-by-case' **analytic continuation** for decay products with an _equal_ mass, {class}`.EqualMassPhaseSpaceFactor`:

In [ ]:
from ampform.dynamics import EqualMassPhaseSpaceFactor

rho_ac = EqualMassPhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho_ac: rho_ac.evaluate()}))

with

In [ ]:
from ampform.dynamics import PhaseSpaceFactorAbs

rho_hat = PhaseSpaceFactorAbs(s, m_a, m_b)
Math(aslatex({rho_hat: rho_hat.evaluate()}))

(Mind the absolute value.)

### 5) Chew-Mandelstam for $S$-waves

A {class}`.PhaseSpaceFactorSWave` that uses {func}`.chew_mandelstam_s_wave`:

In [ ]:
from ampform.dynamics import PhaseSpaceFactorSWave

rho_cm = PhaseSpaceFactorSWave(s, m_a, m_b)
Math(aslatex({rho_cm: rho_cm.evaluate()}))

## Cut structure

When analytically continued into the complex plane, the breakup momentum and phase space factor exhibit distinct cut structures, depending on the definition of the square root in the numerator. The separated square root definition,

$$
\begin{aligned}
\rho_\alpha(s) &= \frac{\sqrt{s-(m_{1,\alpha}-m_{2,\alpha})^2}\sqrt{s-(m_{1,\alpha}+m_{2,\alpha})^2}}{s} \\
q_\alpha(s) &= \frac{\sqrt{s-(m_{1,\alpha}-m_{2,\alpha})^2}\sqrt{s-(m_{1,\alpha}+m_{2,\alpha})^2}}{2\sqrt{s}} \,,
\end{aligned}
$$

leads to a cleaner cut structure than

$$
\begin{aligned}
\rho_\alpha(s) &= \frac{2q_\alpha(s)}{\sqrt{s}} = \frac{\sqrt{(s-(m_{1,\alpha}-m_{2,\alpha})^2) (s-(m_{1,\alpha}+m_{2,\alpha})^2})}{s} \\
q_\alpha(s) &= \frac{\sqrt{(s-(m_{1,\alpha}-m_{2,\alpha})^2)(s-(m_{1,\alpha}+m_{2,\alpha})^2})}{2\sqrt{s}} \,.
\end{aligned}
$$

Here we investigate the cut structure of each of these definitions using AmpForm and SymPy.

In [ ]:
@unevaluated
class PhaseSpaceFactorOld(sp.Expr):
    s: Any
    m1: Any
    m2: Any

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q_squared = BreakupMomentumSquared(s, m1, m2)
        return 2 * sp.sqrt(q_squared) / sp.sqrt(s)

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s = self.args[0]
        s_latex = printer._print(s)
        return Rf"\rho\left({s_latex}\right)"


m1, m2 = sp.symbols("m1 m2")

In [ ]:
q = BreakupMomentum(s, m1, m2)
Math(aslatex({q: q.doit()}))

In [ ]:
rho = PhaseSpaceFactor(s, m1, m2)
Math(aslatex({rho: rho.doit()}))

In [ ]:
q = BreakupMomentumSquared(s, m1, m2)
Math(aslatex({q: sp.sqrt(q).doit()}))

In [ ]:
rho = PhaseSpaceFactorOld(s, m1, m2)
Math(aslatex({rho: rho.doit()}))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats("svg")

x_min, x_max = 0, +1.5
y_max = 0.7
z_max = 0.5
X, Y = np.meshgrid(
    np.linspace(x_min, x_max, num=500),
    np.linspace(-y_max, +y_max, num=300),
)
S = X + 1j * Y
ϵi = 1e-7j

parameters = {m1: 0.2, m2: 0.6}
thr_neg = (parameters[m1] - parameters[m2]) ** 2
thr_pos = (parameters[m1] + parameters[m2]) ** 2

rho_func = sp.lambdify(s, sp.I * PhaseSpaceFactor(s, m1, m2).doit().subs(parameters))
rho_old_func = sp.lambdify(
    s, sp.I * PhaseSpaceFactorOld(s, m1, m2).doit().subs(parameters)
)
q_func = sp.lambdify(s, BreakupMomentum(s, m1, m2).doit().subs(parameters))
q_old_func = sp.lambdify(
    s, sp.sqrt(BreakupMomentumSquared(s, m1, m2)).doit().subs(parameters)
)

fig, axes = plt.subplots(figsize=(9, 7), ncols=2, nrows=2, sharey=True)
fig.patch.set_facecolor("none")
ax1, ax2, ax3, ax4 = axes.flatten()
for ax in axes.flatten():
    ax.patch.set_facecolor("none")
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.set_xlabel(R"$\mathrm{Re}\,s$", labelpad=-12)
    ax.set_xticks([0])
    ax.hlines(0, thr_neg, thr_pos, color="black", lw=1.5, zorder=10)
    ax.scatter([thr_neg, thr_pos], [0, 0], color="black", s=25, zorder=10)
ax1.set_ylabel(R"$\mathrm{Im}\,s$")
ax1.set_ylim(-y_max, +y_max)
ax1.set_yticks([0])

ax3.set_ylabel(R"$\mathrm{Im}\,s$")
ax3.set_ylim(-y_max, +y_max)
ax3.set_yticks([0])
style = dict(
    cmap="coolwarm",
    rasterized=True,
    vmin=-z_max,
    vmax=+z_max,
    zorder=-10,
)
mesh = ax1.pcolormesh(X, Y, rho_func(S).real, **style)
cbar = fig.colorbar(mesh, ax=ax1, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Re}\,i\rho$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
mesh = ax2.pcolormesh(X, Y, q_func(S).imag, **style)
cbar = fig.colorbar(mesh, ax=ax2, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Im}\,q$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
mesh = ax3.pcolormesh(X, Y, rho_old_func(S).real, **style)
cbar = fig.colorbar(mesh, ax=ax3, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Re}\,i\rho$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
mesh = ax4.pcolormesh(X, Y, q_old_func(S).imag, **style)
cbar = fig.colorbar(mesh, ax=ax4, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Im}\,q$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
ax1.plot(X[0], rho_func(X[0] + ϵi).real, c="darkblue")
ax1.plot(X[0], rho_func(X[0] + ϵi).imag, c="darkgreen")
ax2.plot(X[0], q_func(X[0] + ϵi).real, c="darkblue")
ax2.plot(X[0], q_func(X[0] + ϵi).imag, c="darkgreen")
ax3.plot(X[0], rho_old_func(X[0] + ϵi).real, c="darkblue")
ax3.plot(X[0], rho_old_func(X[0] + ϵi).imag, c="darkgreen")
ax4.plot(X[0], q_old_func(X[0] + ϵi).real, c="darkblue")
ax4.plot(X[0], q_old_func(X[0] + ϵi).imag, c="darkgreen")
ax1.text(
    0.99,
    0.48,
    "real",
    c="darkblue",
    transform=ax1.transAxes,
    ha="right",
    va="top",
)
ax1.text(
    0.99,
    0.97,
    "imag",
    c="darkgreen",
    transform=ax1.transAxes,
    ha="right",
    va="top",
)
ax1.set_title("Phase space factor double sqrt")
ax2.set_title("Break-up momentum factor double sqrt")
ax3.set_title("Phase space factor single sqrt")
ax4.set_title("Break-up momentum factor single sqrt")
fig.tight_layout()
plt.show()

:::{note}
When defining the break-up momentum and the phasespace factor with a single square root in the numerator a more complex cut structure emerges when continuing the functions into the complex $s$-plane.
:::

## Dispersion integral

To get an analytic phasespace factor for higher angular momenta, the one has to compute the dispersion integral. According to [PDG, Rev. Resonances](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15) the once-substracted dispersion integral is given by:

$$
\Sigma_a(s+0 i)=\frac{s-s_{\mathrm{thr}_a}}{\pi} \int_{s_{\mathrm{thr}_a}}^{\infty} \frac{\rho_a\left(s^{\prime}\right) n_a^2\left(s^{\prime}\right)}{\left(s^{\prime}-s_{\mathrm{thr}_a}\right)\left(s^{\prime}-s-i 0\right)} \mathrm{d} s^{\prime}
$$

In [ ]:
integral_expr = DispersionIntegral(s, m1, m2, L)
integral_expr.doit(deep=False)

In [ ]:
integral_s_wave_func = sp.lambdify(
    [s, m1, m2, integral_expr.epsilon],
    integral_expr.subs(L, 0).doit(),
)
integral_s_wave_func = np.vectorize(integral_s_wave_func)
integral_p_wave_func = sp.lambdify(
    [s, m1, m2, integral_expr.epsilon],
    integral_expr.subs(L, 1).doit(),
)
integral_p_wave_func = np.vectorize(integral_p_wave_func)

In [ ]:
m1_val = parameters[m1]
m2_val = parameters[m2]
s_values = np.linspace(-0.15, 1.4, num=200)
s_wave_values = integral_s_wave_func(s_values, m1_val, m2_val, epsilon=1e-5)
p_wave_values = integral_p_wave_func(s_values, m1_val, m2_val, epsilon=1e-5)

In [ ]:
l_val = [0, 1]
s_values = np.linspace(-0.15, 1.7, num=200)
fig, axes = plt.subplots(figsize=(6, 7), nrows=2, sharex=True)
fig.patch.set_facecolor("none")
ax1, ax2 = axes
fig.suptitle(f"Symbolic dispersion integrals for $m_1={m1_val:.2f}, m_2={m2_val:.2f}$")
for ax in axes:
    ax.axhline(0, linewidth=0.5, c="black")
    ax.axvline(thr_pos, linestyle="--", color="red", alpha=0.7)
    ax.patch.set_facecolor("none")
    ax.set_title(f"$L = {l_val}$")
    ax.set_ylabel(R"$16\pi \; \Sigma(s)$")
axes[-1].set_xlabel("$s$ (GeV$^2$)")

ax1.set_title("$S$-wave ($L=0$)")
ax1.plot(s_values, 16 * np.pi * s_wave_values.real, label="real")
ax1.plot(s_values, 16 * np.pi * s_wave_values.imag, label="imaginary")

ax2.set_title("$P$-wave ($L=1$)")
ax2.plot(s_values, 16 * np.pi * p_wave_values.real, label="real")
ax2.plot(s_values, 16 * np.pi * p_wave_values.imag, label="imaginary")

ax1.legend(framealpha=0)
fig.tight_layout()
plt.show()

## Interactive visualization

```{autolink-skip}
```

In [ ]:
%matplotlib widget

In [ ]:
import symplot
from ampform.sympy.math import ComplexSqrt

m = sp.Symbol("m", nonnegative=True)
rho_c = PhaseSpaceFactorComplex(m**2, m_a, m_b)
rho_cm = PhaseSpaceFactorSWave(m**2, m_a, m_b)
rho_ac = EqualMassPhaseSpaceFactor(m**2, m_a, m_b)
np_rho_c, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho_c.doit())
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_ac.doit())
np_rho_cm = sp.lambdify((m, m_a, m_b), rho_cm.doit())
np_breakup_momentum = sp.lambdify(
    (m, m_a, m_b),
    2 * ComplexSqrt(q_squared.subs(s, m**2).doit()),
)

{{ run_interactive }}

In [ ]:
plot_domain = np.linspace(0, 3, 500)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.3,
    m_b=0.75,
)

In [ ]:
import mpl_interactions.ipyplot as iplt

import symplot

fig, axes = plt.subplots(
    ncols=2,
    nrows=2,
    figsize=[8, 8],
    sharex=True,
    sharey=True,
)
fig.canvas.footer_visible = False
fig.canvas.header_visible = False
fig.canvas.toolbar_visible = False

(ax_q, ax_rho), (ax_rho_ac, ax_rho_cm) = axes
for ax in [ax_q, ax_rho, ax_rho_cm, ax_rho_ac]:
    ax.set_xlabel("$m$")
    ax.set_yticks([])
for ax in [ax_rho_cm, ax_rho_ac]:
    ax.set_yticks([])

ylim = (-0.1, 1.4)


def func_imag(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).imag


def func_real(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).real


q_math = ComplexSqrt(sp.Symbol("q^2")) / (8 * sp.pi)
ax_q.set_title(f"${sp.latex(q_math)}$")
controls = iplt.plot(
    plot_domain,
    func_real(np_breakup_momentum),
    label="real",
    **sliders,
    ylim=ylim,
    ax=ax_q,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_breakup_momentum),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_q,
    alpha=0.7,
)

ax_rho.set_title(f"${sp.latex(rho_c)}$")
iplt.plot(
    plot_domain,
    func_real(np_rho_c),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_c),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho,
    alpha=0.7,
)

ax_rho_ac.set_title(R"equal mass $\rho^\mathrm{eq}(m^2)$")
iplt.plot(
    plot_domain,
    func_real(np_rho_ac),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_ac,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_ac),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_ac,
    alpha=0.7,
)

ax_rho_cm.set_title(R"Chew-Mandelstam $\rho^\mathrm{CM}(m^2)$")
iplt.plot(
    plot_domain,
    func_real(np_rho_cm),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_cm,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_cm),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_cm,
    alpha=0.7,
)

fig.tight_layout()
plt.legend(loc="upper right")
plt.show()

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG, display

    output_file = "analytic-continuation.svg"
    plt.savefig(output_file)
    display(SVG(output_file))